In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [57]:
# set parameters
σ = 5      # σ > 1
τ = 0.1    # τ > 0
da = 0.1   # 0 < da < 1
db = 0.1   # 0 < db < 1
κ = 1      # κ > 0
pr = 0.5   # pr > 0
cbar = 0   # optional
γ = 5      # gamma >= 0, != 1

# shock parameters (all >= 1)
a = 1
b = 19
αa = 1
βa = 19
αb = 1
βb = 19

# income/endowment
ma = 10
mb = 10
mc = 10

# tariffs and subsidies
tfa = 0
tfb = 0
tfc = 0
tia = 0.24
tib = 0.24
sfa = 0
sfb = 0
sia = 0
sib = 0
ea = 0    # 0 <= ea < 1
eb = 0    # 0 <= eb < 1

# export taxes
txfa = 0
txfb = 0
txia = 0
txib = 0

# include unexpected profits in welfare
# 1 = yes, 0 = no
profs = 1

# number of samples
nsamp = 30

# number of iterations (per sample)
niter = 10000 # suggest >= 10,000

In [59]:
# some preliminary calculations to save space
eϕa = (1 - a/(a + b)) * (1 - αa/(αa + βa)) # expected prob of successful production in country A
eϕb = (1 - a/(a + b)) * (1 - αb/(αb + βb)) # in country B

# input producers form expectations, decide whether to enter

# input prices
pia = σ/(σ - 1)*(cbar - sia + pr/eϕa)
pib = σ/(σ - 1)*(cbar - sib + pr/eϕb)

# input producer entry process
na = 1 # initialise number of firms at 1
nb = 1

max_iter_entry = 1000 # define maximum iterations to prevent infinite loop
iter_entry = 0 # initialise the iteration counter for firm entry

while iter_entry < max_iter_entry:
    
    if na > 0 and nb > 0:
        pfa = (pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))**(1/(1 - σ))
        pfb = ((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))**(1/(1 - σ))
        iaa_share = pia**(1 - σ)/(pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))
        iab_share = (pia*(1 + tib + txia + τ))**(1 - σ)/((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))
        iba_share = (pib*(1 + tia + txib + τ))**(1 - σ)/(pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))
        ibb_share = pib**(1 - σ)/((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))
    else:
        print("Error: Insufficient entry.")
        break
    
    # quantities
    ## final good quantities
    xfaa = pfa**(-σ)*ma/(pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))
    xfba = (pfb*(1 + tfa + txfb + τ))**(-σ)*ma/(pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))
    xfab = (pfa*(1 + tfb + txfa + τ))**(-σ)*mb/((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))
    xfbb = pfb**(-σ)*mb/((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))
    xfac = ((pfa*(1 + tfc + txfa + τ))**(-σ)*mc/
            ((pfa*(1 + tfc + txfa + τ))**(1 - σ) + (pfb*(1 + tfc + txfb + τ))**(1 - σ)))
    xfbc = ((pfb*(1 + tfc + txfb + τ))**(-σ)*mc/
            ((pfa*(1 + tfc + txfa + τ))**(1 - σ) + (pfb*(1 + tfc + txfb + τ))**(1 - σ)))
    
    ## input quantities
    xfa_revenue = pfa*(xfaa + xfab + xfac)
    xfb_revenue = pfb*(xfba + xfbb + xfbc)
    
    xiaa = (xfa_revenue*iaa_share)/pia
    xiab = (xfb_revenue*iab_share)/(pia*(1 + tib + txia + τ))
    xiba = (xfa_revenue*iba_share)/(pib*(1 + tia + txib + τ))
    xibb = (xfb_revenue*ibb_share)/pib
    xia = xiaa + xiab
    xib = xiba + xibb
    
    # find quantity of raw materials to meet expected demand
    if na > 0:
        ra = xia/((1 - da)*na)
    else: ra = 0

    if nb > 0:
        rb = xib/((1 - db)*nb)
    else: rb = 0
    
    # calculate expected profits at that quantity
    epa = pia*ra*eϕa + sia*ra*eϕa - cbar*ra*eϕa - pr*ra - κ*(1 - ea)
    epb = pib*rb*eϕb + sib*rb*eϕb - cbar*rb*eϕb - pr*rb - κ*(1 - eb)
    
    # if positive expected profits, more firms enter
    if epa > 0:
        na = na + 1
    if epb > 0:
        nb = nb + 1

    # if negative profits, subtract one firm and stop
    if epa < 0 and epb < 0:
        na = na - 1
        nb = nb - 1

        # if no firms enter, set number to zero
        if na < 0: 
            na = 0
        if nb < 0: 
            nb = 0
            
        break

    iter_entry = iter_entry + 1
    
    if iter_entry >= max_iter_entry:
        print("Error: Reached maximum iterations, exiting loop.")
        break

# raw materials sourced

if na > 0:
    ra = xia/(eϕa*na)
else:
    ra = 0

if nb > 0:
    rb = xib/(eϕb*nb)
else: rb = 0

# update expected prices and quantities to reflect entry decisions

if na > 0 and nb > 0:
    pfa = (pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))**(1/(1 - σ))
    pfb = ((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))**(1/(1 - σ))
    iaa_share = pia**(1 - σ)/(pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))
    iab_share = (pia*(1 + tib + txia + τ))**(1 - σ)/((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))
    iba_share = (pib*(1 + tia + txib + τ))**(1 - σ)/(pia**(1 - σ) + (pib*(1 + tia + txib + τ))**(1 - σ))
    ibb_share = pib**(1 - σ)/((pia*(1 + tib + txia + τ))**(1 - σ) + pib**(1 - σ))
else:
    print("Error: Insufficient entry.")

# quantities
## final good quantities
xfaa = pfa**(-σ)*ma/(pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))
xfba = (pfb*(1 + tfa + txfb + τ))**(-σ)*ma/(pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))
xfab = (pfa*(1 + tfb + txfa + τ))**(-σ)*mb/((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))
xfbb = pfb**(-σ)*mb/((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))
xfac = ((pfa*(1 + tfc + txfa + τ))**(-σ)*mc/
        ((pfa*(1 + tfc + txfa + τ))**(1 - σ) + (pfb*(1 + tfc + txfb + τ))**(1 - σ)))
xfbc = ((pfb*(1 + tfc + txfb + τ))**(-σ)*mc/
        ((pfa*(1 + tfc + txfa + τ))**(1 - σ) + (pfb*(1 + tfc + txfb + τ))**(1 - σ)))

## input quantities
xfa_revenue = pfa*(xfaa + xfab + xfac)
xfb_revenue = pfb*(xfba + xfbb + xfbc)

xiaa = (xfa_revenue*iaa_share)/pia
xiab = (xfb_revenue*iab_share)/(pia*(1 + tib + txia + τ))
xiba = (xfa_revenue*iba_share)/(pib*(1 + tia + txib + τ))
xibb = (xfb_revenue*ibb_share)/pib
xia = xiaa + xiab
xib = xiba + xibb

# model as it happens

# empty lists to store results
iterations = []
wa_values = []
wb_values = []
wc_values = []
wsum_values = []
cva_values = []
cvb_values = []
cvc_values = []
cvtot_values = []

for i in range(nsamp):
    
    for j in range(niter):

        # draw shock probabilities
        da = stats.beta.rvs(αa, βa)
        db = stats.beta.rvs(αb, βb)
        dk = stats.beta.rvs(a, b)
    
        # input sector
        surviveda = np.random.binomial(na, (1 - da) * (1 - dk))
        survivedb = np.random.binomial(nb, (1 - db) * (1 - dk))
        xia_actual = surviveda*ra
        xib_actual = survivedb*rb
        
        xiaa_actual = xia_actual*iaa_share
        xiab_actual = xia_actual*iab_share
        xiba_actual = xib_actual*iba_share
        xibb_actual = xib_actual*ibb_share
        
        # assume difference is allocated across customers
        # proportionately to expected sales
        faa_share = xfaa/(xfaa + xfab + xfac)
        fab_share = xfab/(xfaa + xfab + xfac)
        fac_share = xfac/(xfaa + xfab + xfac)
        fba_share = xfba/(xfba + xfbb + xfbc)
        fbb_share = xfbb/(xfba + xfbb + xfbc)
        fbc_share = xfbc/(xfba + xfbb + xfbc)
        
        # final sector
        xfaa_actual = (xiaa_actual**((σ - 1)/σ) + xiba_actual**((σ - 1)/σ))**(σ/(σ - 1)) * faa_share
        xfab_actual = (xiaa_actual**((σ - 1)/σ) + xiba_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fab_share
        xfac_actual = (xiaa_actual**((σ - 1)/σ) + xiba_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fac_share
        
        xfba_actual = (xiab_actual**((σ - 1)/σ) + xibb_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fba_share
        xfbb_actual = (xiab_actual**((σ - 1)/σ) + xibb_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fbb_share
        xfbc_actual = (xiab_actual**((σ - 1)/σ) + xibb_actual**((σ - 1)/σ))**(σ/(σ - 1)) * fbc_share
    
        # actual average profits
        if na > 0:
            πa = (pia*(xia_actual)/na + sia*(xia_actual)/na - cbar*(xia_actual)/na - pr*ra - κ*(1 - ea))
        else:
            πa = 0

        if nb > 0:
            πb = (pib*(xib_actual)/nb + sib*(xib_actual)/nb - cbar*(xib_actual)/nb - pr*rb - κ*(1 - eb))
        else:
            πb = 0
        
        # resilience indicators
        
        ## welfare in state s
        if surviveda + survivedb > 0: # welfare is undefined if there is no production
            
            ### 'utility' component
            util_as = ((xfaa_actual**((σ - 1)/σ) + xfba_actual**((σ - 1)/σ))**(σ/(σ - 1)))**(1 - γ)
            util_bs = ((xfab_actual**((σ - 1)/σ) + xfbb_actual**((σ - 1)/σ))**(σ/(σ - 1)))**(1 - γ)
            util_cs = ((xfac_actual**((σ - 1)/σ) + xfbc_actual**((σ - 1)/σ))**(σ/(σ - 1)))**(1 - γ)

        else:
            util_as = 0
            util_bs = 0
            util_cs = 0

        ### 'revenue' component
        rev_as = ((tfa*pfb*xfba_actual + tia*pib*xiba_actual + 
                   txfa*pfa*(xfab_actual + xfac_actual) + txia*pia*xiab_actual - 
                   sia*xia_actual - ea*κ*na + profs*πa*na) /
                    ((pfa**(1 - σ) + (pfb*(1 + tfa + txfb + τ))**(1 - σ))**(1/(1 - σ))))

        rev_bs = ((tfb*pfa*xfab_actual + tib*pia*xiab_actual + 
                   txfb*pfb*(xfba_actual + xfbc_actual) + txib*pib*xiba_actual - 
                   sib*xib_actual - eb*κ*nb + profs*πb*nb) /
                    (((pfa*(1 + tfb + txfa + τ))**(1 - σ) + pfb**(1 - σ))**(1/(1 - σ))))

        rev_cs = ((tfc*pfa*xfac_actual + tfc*pfb*xfbc_actual) / 
          (((pfa*(1 + tfc + txfa + τ))**(1 - σ) + 
            (pfb*(1 + tfc + txfb + τ))**(1 - σ))**(1/(1 - σ))))
        
        ## final good production
        qfa = xfaa_actual + xfab_actual + xfac_actual
        qfb = xfba_actual + xfbb_actual + xfbc_actual
        prod_tot = qfa + qfb
        
        ## final good consumption
        consa = xfaa_actual + xfba_actual
        consb = xfab_actual + xfbb_actual
        consc = xfac_actual + xfbc_actual
        cons_tot = consa + consb + consc
        
        iterations.append({
            "util_as": util_as,
            "rev_as": rev_as,
            "util_bs": util_bs,
            "rev_bs": rev_bs,
            "util_cs": util_cs,
            "rev_cs": rev_cs,
            "consa": consa, 
            "consb": consb,
            "consc": consc,
            "cons_tot": cons_tot
        })

    # extracting values for welfare components
    util_as_values = [iteration["util_as"] for iteration in iterations]
    util_bs_values = [iteration["util_bs"] for iteration in iterations]
    util_cs_values = [iteration["util_cs"] for iteration in iterations]
    rev_as_values = [iteration["rev_as"] for iteration in iterations]
    rev_bs_values = [iteration["rev_bs"] for iteration in iterations]
    rev_cs_values = [iteration["rev_cs"] for iteration in iterations]

    # calculating welfare
    wa = np.mean(util_as_values)**(1/(1 - γ)) + np.mean(rev_as_values)
    wb = np.mean(util_bs_values)**(1/(1 - γ)) + np.mean(rev_bs_values)
    wc = np.mean(util_cs_values)**(1/(1 - γ)) + np.mean(rev_cs_values)
    wsum = wa + wb + wc
    
    # extracting values for consa, consb, consc, and cons_tot
    consa_values = [iteration["consa"] for iteration in iterations]
    consb_values = [iteration["consb"] for iteration in iterations]
    consc_values = [iteration["consc"] for iteration in iterations]
    cons_tot_values = [iteration["cons_tot"] for iteration in iterations]

    # calculating coefficients of variation
    cva = (np.std(consa_values)/np.mean(consa_values))*100
    cvb = (np.std(consb_values)/np.mean(consb_values))*100
    cvc = (np.std(consc_values)/np.mean(consc_values))*100
    cvtot = (np.std(cons_tot_values)/np.mean(cons_tot_values))*100

    # store the calculated values for this sample
    wa_values.append(wa)
    wb_values.append(wb)
    wc_values.append(wc)
    wsum_values.append(wsum)
    cva_values.append(cva)
    cvb_values.append(cvb)
    cvc_values.append(cvc)
    cvtot_values.append(cvtot)

# create a data frame to store the results
df_results = pd.DataFrame({
    "wa": wa_values, 
    "wb": wb_values, 
    "wc": wc_values, 
    "wsum": wsum_values,
    "cva": cva_values, 
    "cvb": cvb_values, 
    "cvc": cvc_values, 
    "cvtot": cvtot_values
})

# print key resililence indicators

print("---------")
print("Country A")
print("---------")
print("n_a:", na)
print("ew_a:", round(df_results["wa"].mean(), 2))
print("ex_a:", round(xfaa + xfba, 2))
print("eq_ia:", round(xiaa + xiab, 2))
print("eq_fa:", round(xfaa + xfab + xfac, 2))
print("cv_xa:", round(df_results["cva"].mean(), 2))

print("---------")
print("Country B")
print("---------")
print("n_b:", nb)
print("ew_b:", round(df_results["wb"].mean(), 2))
print("ex_b:", round(xfab + xfbb, 2))
print("eq_ib:", round(xiba + xibb, 2))
print("eq_fb:", round(xfba + xfbb + xfbc, 2))
print("cv_xb:", round(df_results["cvb"].mean(), 2))

print("---------")
print("Country C")
print("---------")
print("ew_c:", round(df_results["wc"].mean(), 2))
print("ex_c:", round(xfac + xfbc, 2))
print("cv_xc:", round(df_results["cvc"].mean(), 2))

print("-------------")
print("All countries")
print("-------------")
print("n:", na + nb)
print("w:", round(df_results["wsum"].mean(), 2))
print("eq_i:", round(xiaa + xiab + xiba + xibb, 2))
print("eq_f:", round(xfaa + xfba + xfab + xfbb + xfac + xfbc, 2))
print("cv_xtot:", round(df_results["cvtot"].mean(), 2))

print("---------------")
print("Standard errors")
print("---------------")
print("ew_a_sem:", round(stats.sem(df_results["wa"]), 3))
print("cv_xa_sem:", round(stats.sem(df_results["cva"]), 3))
print("ew_b_sem:", round(stats.sem(df_results["wb"]), 3))
print("cv_xb_sem:", round(stats.sem(df_results["cvb"]), 3))
print("ew_c_sem:", round(stats.sem(df_results["wc"]), 3))
print("cv_xc_sem:", round(stats.sem(df_results["cvc"]), 3))
print("ew_sem:", round(stats.sem(df_results["wsum"]), 3))
print("cv_xtot_sem:", round(stats.sem(df_results["cvtot"]), 3))

---------
Country A
---------
n_a: 5
ew_a: 36.6
ex_a: 29.76
eq_ia: 38.48
eq_fa: 43.8
cv_xa: 12.04
---------
Country B
---------
n_b: 5
ew_b: 36.6
ex_b: 29.76
eq_ib: 38.48
eq_fb: 43.8
cv_xb: 12.04
---------
Country C
---------
ew_c: 31.69
ex_c: 28.09
cv_xc: 11.99
-------------
All countries
-------------
n: 10
w: 104.89
eq_i: 76.97
eq_f: 87.6
cv_xtot: 11.99
---------------
Standard errors
---------------
ew_a_sem: 0.024
cv_xa_sem: 0.008
ew_b_sem: 0.02
cv_xb_sem: 0.007
ew_c_sem: 0.018
cv_xc_sem: 0.008
ew_sem: 0.061
cv_xtot_sem: 0.008


In [61]:
iter_entry

5